# `TorchVision`

* `torchvision.datasets`
* `torchvision.models` - pretrained computer vision models
* `torchvision.transforms`- functions for manipulating vision data
* `torch.utils.data.Dataset`- Base dataset class for Pytorch
* `torch.utils.data.DataLoader` - Creates a Python iterable over a dataset

In [ ]:
import torch
from torch import nn
from torch import optim

import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
train_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform =ToTensor(),
    target_transform=None
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data),len(test_data)

In [ ]:
image,label = train_data[0]

In [ ]:
class_names = train_data.classes

In [ ]:
train_data.class_to_idx

In [ ]:
train_data.targets

In [ ]:
train_data[0][0].shape # [Colour_Channel,height,width]

In [ ]:
plt.imshow(image.squeeze())
plt.title(class_names[label])

In [ ]:
plt.imshow(image.squeeze(),cmap="gray")
plt.title(class_names[label])

In [ ]:

fig = plt.figure(figsize=(9,9))
rows,cols = 4,4
for i in range(1,rows*cols+1):
  random_idx = torch.randint(0,len(train_data),size=[1]).item()
  img,label = train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(class_names[label])
  plt.axis(False)

In [ ]:
train_data,test_data

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True
                              )
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False
                             )

train_dataloader,test_dataloader

In [ ]:
print(f"Length of train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

In [ ]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
# torch.manual_seed(42)
random_idx = torch.randint(0,len(train_features_batch),size=[1]).item()
img,label = train_features_batch[random_idx],train_labels_batch[random_idx]
plt.imshow(img.squeeze(),cmap="gray")
plt.title(class_names[label])
plt.axis(False)

print(f"Image Shape: {img.shape}")
print(f"Image Label: {label}")

# Linear Model

In [ ]:
class FashionMNISTModelV0(nn.Module):
  def __init__(self,
               input_shape:int,
               hidden_shape:int,
               output_shape:int
               ):
    super().__init__()

    self.layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,out_features=hidden_shape),
        nn.Linear(in_features=hidden_shape,out_features=output_shape)
    )

  def forward(self,x):
    return self.layer(x)

In [ ]:
torch.manual_seed(42)

model_0 = FashionMNISTModelV0(
    input_shape=784,
    hidden_shape=10,
    output_shape=len(class_names)
).to("cpu")

print(model_0)

In [ ]:
def accuracy_fn(y_true,y_pred):
  correct = torch.eq(y_true,y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_0.parameters(),lr=0.1)

In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float,
                     end: float,
                     device: torch.device):
  total_time = end - start
  print(f"Train time on {device}: {total_time:.3f} seconds")
  return total_time

In [ ]:
start_time = timer()

end_time = timer()
print_train_time(start=start_time,end=end_time,device='cpu')

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n")
  train_loss,train_acc = 0,0
  for batch, (X,y) in enumerate(train_dataloader):
    # if batch % 1875 == 0:
    #   print("Ended")
    #   print(batch)
    y_pred = model_0(X)
    loss = loss_fn(y_pred,y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if batch % 600 == 0:
      print(f"Looked at {batch * len(X)} / {len(train_dataloader.dataset)} samples.")

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)
  test_loss, test_acc = 0,0
  model_0.eval()
  with torch.inference_mode():
    for X_t,y_t in test_dataloader:
      test_pred = model_0(X_t)
      test_loss += loss_fn(test_pred,y_t)

      test_acc += accuracy_fn(y_true=y_t,y_pred=test_pred.argmax(dim=1))
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  print(f"\nTrain loss: {train_loss:.4f} | Test acc: {train_acc} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}%")

train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu,end=train_time_end_on_cpu,device=str(next(model_0.parameters()).device))

In [ ]:
torch.manual_seed(42)
def eval_mode(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn
              ):
  loss,acc = 0,0
  model.eval()
  with torch.inference_mode():
    for X,y in tqdm(data_loader):
      y_pred = model(X)

      loss += loss_fn(y_pred,y)
      acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"model_name": model.__class__.__name__,
          "model_loss":loss,
          "model_acc":acc}

In [ ]:
model_0_results = eval_mode(model=model_0,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)

In [ ]:
model_0_results

# Non-Linear Model

In [ ]:
class FashionMNISTModelV1(nn.Module):
  def __init__(self,input_shape,hidden_shape,output_shape):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Flatten(), # Add Flatten layer here
        nn.Linear(in_features=input_shape,out_features=hidden_shape),
        nn.ReLU(),
        nn.Linear(in_features=hidden_shape,out_features=output_shape)
    )
  def forward(self,x):
    return self.layer(x)

In [ ]:
model_1 = FashionMNISTModelV1(
                              input_shape=784,
                              hidden_shape=30,
                              output_shape=len(class_names)
                              )
model_1

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_1.parameters(),lr=0.1)


In [ ]:
torch.manual_seed(42)
from tqdm.auto import tqdm
epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}")
  train_loss, train_acc = 0,0
  model_1.train()
  for batch, (X,y) in enumerate(train_dataloader):
    y_pred = model_1(X)
    loss = loss_fn(y_pred,y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  model_1.eval()
  with torch.inference_mode():
    test_acc,test_loss = 0,0
    for X_t,y_t in test_dataloader:
      test_pred = model_1(X_t)
      test_loss += loss_fn(test_pred,y_t)
      test_acc += accuracy_fn(y_true=y_t,y_pred=test_pred.argmax(dim=1))
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  print(f"\nTrain loss: {train_loss:.4f} | Train acc: {train_acc} | Test loss: {test_loss:.4f}, Test acc: {test_acc:.4f}%")

In [ ]:
model_1_results = eval_mode(model=model_1,
                            data_loader=test_dataloader,
                            loss_fn=loss_fn,
                            accuracy_fn=accuracy_fn
                            )

In [ ]:
model_0_results,"="*35,model_1_results

# Convolutional Neural Network

In [ ]:
class FashionMNISTModelV2(nn.Module):
  def __init__(self,input_shape:int,
               hidden_shape:int,
               output_shape:int):
    super().__init__()
    self.cnn_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=1
                ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=1
                  ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.cnn_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=1
                ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=1
                  ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_shape*49,
                  out_features=output_shape
                  )
    )

  def forward(self,x):
    x = self.cnn_1(x)
    # print(f"Output shape of cnn layer 1: {x.shape}")
    x = self.cnn_2(x)
    # print(f"Output shape of cnn layer 1: {x.shape}")
    x = self.classifier(x)
    # print(f"Output shape of classifier: {x.shape}")
    return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape=1, # gray images
                              hidden_shape=10,
                              output_shape=len(class_names)
                              )
model_2

In [ ]:
model_2(image.unsqueeze(dim=0))

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_2.parameters(),lr=0.1)

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn
               ):
  train_loss, train_acc = 0,0

  model.train()

  for batch, (X,y) in enumerate(data_loader):

    y_pred = model(X)
    loss = loss_fn(y_pred,y)
    train_loss += loss
    train_acc += accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train Loss: {train_loss:.5f} | Train acc: {train_acc:.2f}%")

def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn
              ):
  model_1.eval()
  test_acc,test_loss = 0,0
  with torch.inference_mode():
    for X_t,y_t in data_loader:
      test_pred = model(X_t)
      test_loss += loss_fn(test_pred,y_t)
      test_acc += accuracy_fn(y_true=y_t,y_pred=test_pred.argmax(dim=1))
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
  print(f"Train Loss: {test_loss:.5f} | Train acc: {test_acc:.2f}%")

In [ ]:
torch.manual_seed(42)

from timeit import default_timer as timer
# def print_train_time(start: float,
#                      end: float,
#                      device: torch.device):
#   total_time = end - start
#   print(f"Train time on {device}: {total_time:.3f} seconds")
#   return total_time
train_epoch_start_model_2 = timer()
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}")
  train_step(model=model_2,
             data_loader=train_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             optimizer=optimizer
             )
  test_step(model=model_2,
            data_loader=test_dataloader,
            accuracy_fn=accuracy_fn,
            loss_fn=loss_fn
            )
train_epoch_end_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_epoch_start_model_2,end=train_epoch_end_model_2,device=str(next(model_2.parameters()).device))

In [ ]:
model_2_results = eval_mode(model=model_2,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)

In [ ]:
model_0_results,"="*35,model_1_results,"="*35,model_2_results

In [ ]:
import pandas as pd

compare_results = pd.DataFrame([model_0_results,
                                model_1_results,
                                model_2_results])
compare_results

In [ ]:
def make_predictions(model:torch.nn.Module,
                     data: list):
  pred_probs = []
  model.eval()
  with torch.inference_mode():
    for sample in data:
      sample = torch.unsqueeze(sample,dim=0)
      pred_logit = model_2(sample)
      pred_prob = torch.softmax(pred_logit.squeeze(),dim=0)

      pred_probs.append(pred_prob)
  return torch.stack(pred_probs)

In [ ]:
import random
random.seed(42)
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_data),k=9):
  test_samples.append(sample)
  test_labels.append(label)

test_samples[0].shape

In [ ]:
plt.imshow(test_samples[0].squeeze(), cmap="gray")
plt.title(class_names[test_labels[0]])

In [ ]:
pred_probs = make_predictions(model=model_2,
                              data=test_samples)

In [ ]:
pred_probs[:2]

In [ ]:
pred_classes = pred_probs.argmax(dim=1)
print(pred_classes)
print(test_labels)

In [ ]:
from tqdm.auto import tqdm

y_preds = []
y_true = []
model_2.eval()
with torch.inference_mode():
  for X,y in tqdm(test_dataloader,desc="Making predictions..."):
    y_true.append(y)
    y_logits = model_2(X)
    test_loss = loss_fn(y_logits,y)
    y_pred = torch.softmax(y_logits.squeeze(),dim=0).argmax(dim=1)
    y_preds.append(y_pred)

In [ ]:
try:
    import torchmetrics
    import mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend version should be 0.19.0 or higher"
except (ImportError, AssertionError):
    %pip install torchmetrics -U mlxtend
    import torchmetrics
    import mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")


In [ ]:
# Concatenate list of tensors into a single tensor
y_pred_tensor = torch.cat(y_preds)
y_true_tensor = torch.cat(y_true)
print(y_pred_tensor[:10])
print(y_true_tensor[:10])

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# Setup confusion matrix
confmat = ConfusionMatrix(task='multiclass', num_classes=len(class_names))
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_data.targets
                         )
fig,axis = plot_confusion_matrix(
    conf_mat = confmat_tensor.numpy(),
    class_names=class_names,
    figsize=(8,4)
)

In [ ]:
print(confmat_tensor)

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)
MODEL_NAME = "02_pytorch_computer_vision.pth"
MODLE_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODLE_SAVE_PATH}")
torch.save(obj=model_2.state_dict(),
           f=MODLE_SAVE_PATH)

In [ ]:
MODEL_PATH = Path("models00")
MODEL_PATH.mkdir(parents=False,
                 exist_ok=True)

In [ ]:
loaded_model = FashionMNISTModelV2(input_shape=1,
                                           hidden_shape=10,
                                           output_shape=len(class_names)
                                           )

loaded_model.load_state_dict(torch.load(f=MODLE_SAVE_PATH))

In [ ]:
print(loaded_model)

In [ ]:
model_2_results

In [ ]:
torch.manual_seed(42)

loaded_model = eval_mode(
    model=loaded_model,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)
loaded_model